In [1]:
import networkx as nx

In [1]:
def re_triangular_lattice_graph(
    m, n, periodic=False, with_positions=True, create_using=None
):
    from numpy import sqrt
    from networkx import empty_graph, NetworkXError, set_node_attributes
    from networkx.algorithms.minors import contracted_nodes
    H = empty_graph(0, create_using)
    if n == 0 or m == 0:
        return H
    if periodic:
        if n < 5 or m < 3:
            msg = f"m > 2 and n > 4 required for periodic. m={m}, n={n}"
            raise NetworkXError(msg)

    N = (n + 1) // 2  # number of nodes in row
    rows = range(m + 1)
    cols = range(N + 1)
    # Make grid
    print("add edge 1")
    H.add_edges_from(((i, j), (i + 1, j)) for j in rows for i in cols[:N])
    print("add edge 2")
    H.add_edges_from(((i, j), (i, j + 1)) for j in rows[:m] for i in cols)
    # add diagonals
    print("add edge 3")
    H.add_edges_from(((i, j), (i + 1, j + 1)) for j in rows[1:m:2] for i in cols[:N])
    print("add edge 4")
    H.add_edges_from(((i + 1, j), (i, j + 1)) for j in rows[:m:2] for i in cols[:N])
    # identify boundary nodes if periodic
    print("periodicity")
    if periodic is True:
        for i in cols:
            H = contracted_nodes(H, (i, 0), (i, m))
        for j in rows[:m]:
            H = contracted_nodes(H, (0, j), (N, j))
    elif n % 2:
        # remove extra nodes
        H.remove_nodes_from((N, j) for j in rows[1::2])

    # Add position node attributes
    if with_positions:
        ii = (i for i in cols for j in rows)
        jj = (j for i in cols for j in rows)
        xx = (0.5 * (j % 2) + i for i in cols for j in rows)
        h = sqrt(3) / 2
        if periodic:
            yy = (h * j + 0.01 * i * i for i in cols for j in rows)
        else:
            yy = (h * j for i in cols for j in rows)
        pos = {(i, j): (x, y) for i, j, x, y in zip(ii, jj, xx, yy) if (i, j) in H}
        set_node_attributes(H, pos, "pos")
    return H

In [3]:
G = nx.triangular_lattice_graph(512, 256, periodic=True)